# Proyecto RI

### Dataset obtenido de Kaggle

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import unicodedata
from iswd723 import clean_text, stemming, remove_stopwords, filter_tokens
import iswd723
from collections import Counter

[nltk_data] Downloading package stopwords to C:\Users\Mauricio
[nltk_data]     Morales\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('data/bbc_news.csv', encoding='utf-8')
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [3]:
df['clean_text'] = df['description'].apply(clean_text).apply(remove_stopwords).apply(filter_tokens).apply(stemming)

In [4]:
df['clean_text']

0        ukrainian presid say countri forgiv forget mur...
1        jeremi bowen frontlin irpin resid came russian...
2        one world biggest fertilis firm say conflict c...
3        parent manchest arena bomb youngest victim spe...
4        consum feel impact higher energi cost fuel pri...
                               ...                        
42110    watch highlight wale win dublin aggreg triumph...
42111        two tonn class drug seiz shipment colombia uk
42112    financ secretari shona robison prepar set spen...
42113    wale defeat republ ireland make footbal histor...
42114    three children back belfast model school girl ...
Name: clean_text, Length: 42115, dtype: object

### Creación de indice invertido

In [5]:
indice_inv = iswd723.make_index_inv(df['clean_text'])
indice_inv_tf = {i : {'docs':indice_inv[i], 'freq':len(indice_inv[i])} for i in indice_inv.keys()}

### Modelo binario

In [6]:
vocab = iswd723.build_vocabulary(df['clean_text'])

In [7]:
vocab_map = {word : i for i, word in enumerate(vocab)}

In [ ]:
mat = np.zeros((len(df['clean_text']),len(vocab)), dtype=int) #mat se almacena el vector binario
for key, docs in indice_inv.items():
    mat[docs, [vocab_map[key]]] = 1

In [8]:
tf = df['clean_text'].apply(lambda x: iswd723.get_tf(x, vocab_map, len(vocab)))

In [10]:
tf.values[0]

array([0., 0., 0., ..., 0., 0., 0.], shape=(21357,))

In [11]:
tf_ = np.stack(tf.values)

In [12]:
df_ = iswd723.get_df(tf_)
idf = iswd723.get_idf(df['clean_text'], df_)

In [13]:
tfidf__ = iswd723.get_tfidf(tf_, idf)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tf_idf = vect.fit_transform(df['clean_text'])
mat_tfidf = pd.DataFrame(
    tf_idf.toarray(),
    columns = vect.get_feature_names_out()
)
mat_tfidf.head()

,aa,aaliyah,aamir,aanoch,aardman,aaron,aaronovitch,aastha,ab,abandon,...,zuniga,zunino,zunyou,zurcher,zurich,zuu,zuwara,zverev,zwelithini,zwerner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
vector_tfidf = tf_idf.toarray()